In [1]:
import numpy as np
from msmbuilder.feature_selection import VarianceThreshold
from msmbuilder.decomposition import tICA
from msmbuilder.cluster import MiniBatchKMeans, KMeans, LandmarkAgglomerative, MiniBatchKMedoids, KMedoids
from msmbuilder.msm import MarkovStateModel
from sklearn.pipeline import Pipeline
from glob import glob

/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# variance_cut
# 		 threshold 	 0.0
# tica
# 		 commute_mapping 	 False
# 		 kinetic_mapping 	 True
# 		 lag_time 	 1
# 		 n_components 	 4
# 		 shrinkage 	 None
# cluster
# 		 batch_size 	 100
# 		 compute_labels 	 True
# 		 init 	 k-means++
# 		 init_size 	 None
# 		 max_iter 	 100
# 		 max_no_improvement 	 10
# 		 n_clusters 	 566
# 		 n_init 	 3
# 		 random_state 	 None
# 		 reassignment_ratio 	 0.01
# 		 tol 	 0.0
# 		 verbose 	 0
# msm
# 		 ergodic_cutoff 	 on
# 		 lag_time 	 50
# 		 n_timescales 	 2
# 		 prior_counts 	 0
# 		 reversible_type 	 mle
# 		 sliding_window 	 True
# 		 use_gap 	 None
# 		 verbose 	 True

In [3]:
traj_paths = glob('/home/robert/Datasets/DHFR/train/pp_tor/trajectory-*.npy')

len(traj_paths)

350

In [4]:
trajs = [np.load(x) for x in traj_paths]

In [5]:
selector = VarianceThreshold()
vtrajs = selector.fit_transform(trajs)

In [6]:
len(selector.get_support(indices=True))

632

In [7]:
from sklearn.model_selection import ShuffleSplit

In [8]:
cv = ShuffleSplit(n_splits = 5, test_size=0.2, random_state=1)
for tr_idx, te_idx in cv.split(trajs):
    train = np.array(trajs)[tr_idx]
    selector = VarianceThreshold()
    selector.fit(train)
    print(len(selector.get_support(indices=True)))
    test = np.array(trajs)[te_idx]
    selector = VarianceThreshold()
    selector.fit(test)
    print(len(selector.get_support(indices=True)))
    print()

632
632

632
632

632
632

632
632

632
632



In [9]:
train = [np.array(trajs)[i] for i, _ in cv.split(trajs)]
test = [np.array(trajs)[i] for _, i in cv.split(trajs)]

In [10]:
train_idx = [i for i, _ in cv.split(trajs)]
test_idx = [i for _, i in cv.split(trajs)]

In [20]:
var = VarianceThreshold(threshold=1e-3)
tica=tICA(kinetic_mapping=True, lag_time=76, n_components=9)
cluster = MiniBatchKMeans(n_clusters=777, batch_size=1000)
cluster = LandmarkAgglomerative(n_clusters=777, linkage='ward', n_landmarks=777)
msm = MarkovStateModel(lag_time=50, n_timescales=2, prior_counts=1)

In [21]:
vtrajs = var.fit_transform(train[0])
print(len(var.get_support(indices=True)))
ttrajs = tica.fit_transform(vtrajs)

626


In [22]:
ctrajs = cluster.fit_transform(ttrajs)

/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/robert/anaconda3/envs/science/lib/python3.5/site-packages/numpy/core/_methods.py:68: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to 

In [23]:
mtrajs = msm.fit_transform(ctrajs)

MSM contains 15 strongly connected components above weight=0.02. Component 1 selected, with population 30.969173%


In [24]:
tvtrajs = var.transform(test[0])
tttrajs = tica.transform(tvtrajs)
tctrajs = cluster.transform(tttrajs)

No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to cluster 775
No data points were assigned to cluster 776
No data points were assigned to 

In [25]:
msm.score(tctrajs)

MSM contains 18 strongly connected components above weight=0.02. Component 9 selected, with population 27.139850%


ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
discarded = [x for x in range(777) if x not in msm.mapping_.keys()]
kept = [x for x in range(777) if x  in msm.mapping_.keys()]

In [ ]:
print(len(kept), len(discarded), len(kept)+len(discarded))

In [ ]:
# np.array([item in b for item in a])
discarded_frames = [np.where([item in discarded for item in cluster.labels_[i]])[0] for i in range(len(train[0]))] 
kept_frames = [np.where([item in kept for item in cluster.labels_[i]])[0] for i in range(len(train[0]))] 

In [ ]:
for i in range(175):
    if len(discarded_frames[i])>0:
        print('discarded: ', traj_paths[train_idx[0][i]])
    if len(kept_frames[i])>0:
        print('kept: ', traj_paths[train_idx[0][i]])
